In [1]:
#from transformer import Transformer 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
import os
import gc
import random
import numpy as np
#import polars as pl
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset, Dataset, random_split
import fastai
from fastai.vision.all import *

import math
import copy

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#device = "cpu"


max_seq_len = 206 + 2  # +2 is added to cover start and end token of sequence This will change to 457 in private set  # Set the maximum sequence length to 100
data_preprocessing_done = 0
save_pkl = 0
load_pkl = 0
small_dataset = 0

#Training Data
if small_dataset == 1:
    
    #Uncomment this section when uploading the jupyter notebook
    '''
    df_read = pd.read_csv('/media/spartans/COMMON:/RNA_Project/train_data.csv')
    
    criteria_2A3 = (df_read['experiment_type'] == '2A3_MaP') & (df_read['SN_filter'] == 1)
    criteria_DMS = (df_read['experiment_type'] == 'DMS_MaP') & (df_read['SN_filter'] == 1)

    df_2A3_Filter1 = df_read[criteria_2A3]
    df_2A3_Filter1 = df_2A3_Filter1.head(4)

    df_DMS_Filter1 = df_read[criteria_DMS]
    df_DMS_Filter1 = df_DMS_Filter1.head(4)

   
    df = pd.concat([df_2A3_Filter1, df_DMS_Filter1], ignore_index=True)
    
    df.to_csv('2A3_DMS_With_Filter1_Train_Data.csv', index=False)
    #df_2A3_Filter1.to_csv('2A3_DMS_With_Filter1_Train_Data.csv', index=False)
    '''
    df = pd.read_csv('2A3_DMS_With_Filter1_Train_Data.csv')
    #df = pd.read_csv('DMS_Data.csv')
    #df = pd.read_csv('2A3_Data.csv')
else:
    df = pd.read_csv('/media/spartans/COMMON:/RNA_Project/train_data.csv')
    

#test_df = pd.read_csv('/media/spartans/COMMON:/RNA_Project/test_sequences.csv')


seed=69
os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)

os.environ['TORCH_USE_CUDA_DSA'] = str(1)


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(seed)

In [2]:



#criteria_2A3 = (df_read['experiment_type'] == '2A3_MaP') & (df_read['SN_filter'] == '1')
#criteria_DMS = (df_read['experiment_type'] == 'DMS_MaP') & (df_read['SN_filter'] == '1')

#criteria_2A3 = ((df_read['experiment_type'] == 'DMS_MaP')  & df_read['SN_filter'] == 1 )
#df_2A3_Filter1 = df_read[criteria_2A3]
#df_2A3_Filter1
df

,sequence_id,sequence,experiment_type,dataset_name,reads,signal_to_noise,SN_filter,reactivity_0001,reactivity_0002,reactivity_0003,...,reactivity_error_0197,reactivity_error_0198,reactivity_error_0199,reactivity_error_0200,reactivity_error_0201,reactivity_error_0202,reactivity_error_0203,reactivity_error_0204,reactivity_error_0205,reactivity_error_0206
0,8cdfeef009ea,GGGAACGACUCGAGUAGAGUCGAAAAACGUUGAUAUGGAUUUACUCCGAGGAGACGAACUACCACGAACAGGGGAAACUCUACCCGUGGCGUCUCCGUUUGACGAGUAAGUCCUAAGUCAACAUGCCACGCGGGUCCUUCGGGACCCGCAAAAGAAACAACAACAACAAC,2A3_MaP,15k_2A3,2343,0.944,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,51e61fbde94d,GGGAACGACUCGAGUAGAGUCGAAAAACAUUGAUAUGGAUUUACUCCGAGGAGACGAACUACCACGAACAGGGGAAACUCUACCCGUGGCGUCUCCGUUUGACGAGUAAGUCCUAAGUCAACAUGCACAGCGCUGGGUUCGCCCAGCGCAAAAGAAACAACAACAACAAC,2A3_MaP,15k_2A3,5326,1.933,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25ce8d5109cd,GGGAACGACUCGAGUAGAGUCGAAAAACCUUGAUAUGGAUUUACUCCGAGGAGACGAACUACCACGAACAGGGGAAACUCUACCCGUGGCGUCUCCGUUUGACGAGUAAGUCCUAAGUCAACAUGCAUGGGUCCUCCUUCGGGAGGACCAAAAGAAACAACAACAACAAC,2A3_MaP,15k_2A3,4647,2.347,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,07dcfb6d1965,GGGAACGACUCGAGUAGAGUCGAAAAACUUUGAUAUGGAUUUACUCCGAGGAGACGAACUACCACGAACAGGGGAAACUCUACCCGUGGCGUCUCCGUUUGACGAGUAAGUCCUAAGUCAACAUGCCAGGUAUUGACUUCGGUCAAUACAAAAGAAACAACAACAACAAC,2A3_MaP,15k_2A3,102843,11.824,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,e561cc042a4c,GGGAACGACUCGAGUAGAGUCGAAAAACGAUGAUAUGGAUUUACUCCGAGGAGACGAACUACCACGAACAGGGGAAACUCUACCCGUGGCGUCUCCGUUUGACGAGUAAGUCCUAAGUCAACAUGCAUGCAUGCGGCUUCGGCCGCAUGAAAAGAAACAACAACAACAAC,2A3_MaP,15k_2A3,7665,3.519,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1643675,7951fb2f47f1,GGGAACGACUCGAGUAGAGUCGAAAAGGAGCGUCGUGUCUCUUGUACGUCUCGGUCACAAUACACGGUUUCGUCCGGUGCGUGGCAAUUCGGGGCACAUCAUGUUUUUCGUGCCUGGUGUGACCGCGCAAGGUGCGCGCGGUACGUAUCGAGCAGCGCUCCAAAACUCGCAACUUCGGUUGCGAGAAAAGAAACAACAACAACAAC,DMS_MaP,SL5_M2seq_DMS,37530,7.248,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1643676,e0dc5823e5e1,GGGAACGACUCGAGUAGAGUCGAAAAGGAGCGUCGUGUCUCUUGUACGUCUCGGUCACAAUACACGGUUUCGUCCGGUGCGUGGCAAUUCGGGGCACAUCAUGUUUUUCGUGUCUGGUGUGACCGCGCAAGGUGCGCGCGGUACGUAUCGAGCAGCGCUCCAAAAACAUCGGAUUCGUCCGAUGUAAAAGAAACAACAACAACAAC,DMS_MaP,SL5_M2seq_DMS,337248,17.902,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1643677,0d6036529b42,GGGAACGACUCGAGUAGAGUCGAAAAGGAGCGUCGUGUCUCUUGUACGUCUCGGUCACAAUACACGGUUUCGUCCGGUGCGUGGCAAUUCGGGGCACAUCAUGUUUUUCGUGGAUGGUGUGACCGCGCAAGGUGCGCGCGGUACGUAUCGAGCAGCGCUCCAAAAACCCCGGUUUCGACCGGGGUAAAAGAAACAACAACAACAAC,DMS_MaP,SL5_M2seq_DMS,44053,6.700,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1643678,46d1f07d723b,GGGAACGACUCGAGUAGAGUCGAAAAGGAGCGUCGUGUCUCUUGUACGUCUCGGUCACAAUACACGGUUUCGUCCGGUGCGUGGCAAUUCGGGGCACAUCAUGUUUUUCGUGGGUGGUGUGACCGCGCAAGGUGCGCGCGGUACGUAUCGAGCAGCGCUCCAAAAGAGCUAUCUUCGGAUAGCUCAAAAGAAACAACAACAACAAC,DMS_MaP,SL5_M2seq_DMS,108600,11.716,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-1, -2)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        #print(output)
        return output

In [4]:
class PositionWiseFeedForward1(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))
    
class PositionWiseFeedForward(nn.Module):

    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionWiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        #print(f"x after first linear layer: {x.size()}")
        x = self.relu(x)
        #print(f"x after activation: {x.size()}")
        x = self.dropout(x)
        #print(f"x after dropout: {x.size()}")
        x = self.linear2(x)
        #print(f"x after 2nd linear layer: {x.size()}")
        return x


In [5]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        #print(x.size())
        #print(self.pe[:, :x.size(1)].size())
        #print((x + self.pe[:, :x.size(1)]).size())
        return x + self.pe[:, :x.size(1)]
    

class PositionalEncoding1(nn.Module):

    def __init__(self, d_model, max_sequence_length):
        super().__init__()
        self.max_sequence_length = max_sequence_length
        self.d_model = d_model

    def forward(self, x):
        even_i = torch.arange(0, self.d_model, 2).float()
        denominator = torch.pow(10000, even_i/self.d_model)
        position = torch.arange(self.max_sequence_length).reshape(self.max_sequence_length, 1)
        even_PE = torch.sin(position / denominator)
        odd_PE = torch.cos(position / denominator)
        stacked = torch.stack([even_PE, odd_PE], dim=2)
        PE = torch.flatten(stacked, start_dim=1, end_dim=2)
        print("PE size", PE.size())
        print("x size", x.size())
        print("PE x size", PE[:, :x.size(1)].size())
        print("stacked dim", stacked.size())
        return x + PE[:, :x.size(1)]
    

In [6]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [7]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x
    
    


In [8]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        
        
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.transformer_decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model=d_model, nhead=num_heads),
            num_layers=num_layers
        )
        
        
        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.gelu = nn.GELU()
        self.fc1 = nn.Linear(max_seq_length-2,max_seq_length-2)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool().to(device)
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        #print("forward ",src, src.size())
        print("forward ", tgt, tgt.size())
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        #print("Src_embedded size", src_embedded.size())
        #print("tgt size ", tgt.size())
        #print("TGT  = ", tgt)
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))
        #print("tgt_embedded size", tgt_embedded.size())

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)
        
        # Expand the dimensions to create an initial input for the decoder
        #x = enc_output.unsqueeze(1).expand(-1, self.output_sequence_length, -1)


        #output = self.fc(dec_output)
        #print("Size OUTPUT",output.size())
        
        xx = self.gelu(self.fc(dec_output))
        print("Size xx", xx.size())
        final_out = self.fc1(xx)
        print("Size final_out", final_out.size())
        return final_out
        #include here FC layers with output dimension = input dimension of the sequence (considering padding this will be const number)
        #return output

In [9]:
#where reactivity is NaN,
# have Nan replaced with average of all numbers in the sequence.

#Map sequence letters to numbers.

#Looking at Reads and Signal To Noise ..they appear to be somewhat coorelated.

#Create Dataset function


#In contrastive loss model... you can train based on#
#in a sequence which part is 2D fold and which part is 3D fold
#See notebook https://www.kaggle.com/code/something4kag/ribonanza-3d-coords-prep
#if it can be helpful
# Also analyze and which position in the sequence the fold can occur and which fold ended up happening based on the data

#another method to formulate contrastive learning model is by deferentiating sequences with low SNR and hence reactivity value will not be with high confidence.



In [10]:

#In RNA, the most common base pairings you'll find are between the following nucleotide bases:

#Adenine (A) and Uracil (U): A forms base pairs with U. This is a fundamental pairing and is commonly seen in RNA molecules, particularly in single-stranded regions. It is important for the stability of stem-loop structures and is a key component of RNA secondary structure.

#Guanine (G) and Cytosine (C): G forms base pairs with C, just as it does in DNA. This pairing is less common in RNA secondary structure but is still important for certain RNA molecules. It may be more prevalent in the context of ribozymes and catalytic RNA.

#While A-U and G-C are the primary base pairings in RNA, it's essential to understand that RNA can also exhibit non-canonical or non-standard base pairings, especially in more complex RNA structures. These non-canonical pairings can involve different combinations of A, U, G, and C and are often seen in tertiary structures or specialized RNA molecules with specific functions.

#The prevalence of A-U and G-C base pairings in an RNA molecule can vary depending on its sequence and function. For instance, regions that need to form stable secondary structures often rely on A-U pairings, while regions involved in catalytic activities may include G-C pairs. RNA structures are diverse and can exhibit a wide range of base pairing interactions to achieve their biological functions.



#So a contrastive loss can be formed here where
#AU pair GC pair are strong positive - stable
#AG, UC are less stable
#UG, UA, CG pair are unstable so negative .
#
#In RNA, the stable base pairs among the four nucleotide bases (A, G, U, and C) follow the standard Watson-Crick base-pairing rules. Here are the stable base pairs among these bases:

#GC pairs are more stable than AU pair


#So you can tokenzie these pairs in the transformer model.

In [11]:
#Formulate Convolution NN or Dense Net which takes SNR, Reactivity error and feeds into the final layer of Transformer network
# As an example
#Lowest Reactivity Error and High SNR are  strong positives
# Lowest SNR are strong negative.





#sequence length range from 115 to 206 in train dataset.
#in final it will range from 207 to 457

In [12]:
#In positional encoding
#pos means the position of the word in the sequence.
#i means the index for that particular word vector (index of the dimension)
#dmodel is the dimension for e.g 512 from the example


#For 2 experiment types i.e. DMS and 2A3 you can have start and end  token in a sequence for which data is available.
#.e. start_DMS, end_DMS, start_2A3, end_2A3.




In [13]:
if data_preprocessing_done == 0:
    #Assigning reactivity Columns to react_columns dataframe and replacing NaN with 0.000 

    react_columns = df.iloc[:,7:213].fillna(0.000000)

    #Find mean of reactivity per row
    react_columns = react_columns.applymap(lambda x: 0 if isinstance(x, (float, int)) and x < 0 else x)

    react_columns_mean = react_columns.mean(axis=1)
    #Free up memory
    del(react_columns)

    #Now replace NaN in react_columns with mean_reactivity
    #react_columns_New = df.iloc[:,7:213].fillna(mean_reactivity)

/tmp/ipykernel_247140/3653553058.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  react_columns = react_columns.applymap(lambda x: 0 if isinstance(x, (float, int)) and x < 0 else x)


In [ ]:

if data_preprocessing_done == 0:
    react_columns = df.iloc[:,7:213] #You may have to capture those indices as per sequence length i.e. when it extends to 457

    # Replace NaN values in each row with values from replace_values array
    for index, row in tqdm(react_columns.iterrows(), total=len(react_columns), desc="Processing rows"):
        react_columns.loc[index] = row.fillna(react_columns_mean[index])
    
    react_columns = react_columns.applymap(lambda x: 0 if isinstance(x, (float, int)) and x < 0 else x)


        
        
        

Processing rows:  37%|██████████████████████████▍                                             | 603760/1643680 [01:23<02:25, 7147.57it/s]

In [ ]:
seq_map = {'A':1.0,
           'C':2.0,
           'G':3.0,
           'U':4.0, 
           'M':5.0,  #Start token of DMS exp seq
           'N':6.0,  #End token of DMS exp seq
           'T':7.0,  #Start token of 2A3 exp seq
           'X':8.0,  #End token of 2A3 exp seq
           'Z':0.0   #Padding token
          }


In [ ]:
# Iterate over rows and replace a specific value in all columns


#max_seq_len = 206 #This will change to 457 in private set  # Set the maximum sequence length to 100
if data_preprocessing_done == 0:
    df_sq = copy.deepcopy(df['sequence']) #saving a backup of sequences.
    #react_columns['maped_sequence'] = [' ']*len(react_columns)
    #for index, row in tqdm(df.iterrows(), total=len(df.rows), desc="Processing rows"):
    for index, row in tqdm(react_columns.iterrows(), total=len(react_columns), desc="Processing rows"):
        if df.at[index, 'experiment_type'] == "2A3_MaP":
            concat_seq = 'T' + df.at[index, 'sequence'] + 'X'
        if df.at[index, 'experiment_type'] == "DMS_MaP":
            concat_seq = 'M' + df.at[index, 'sequence'] + 'N'
        padding = "Z"*(max_seq_len - 2 - len(df.at[index, 'sequence']))
        concatenated_seq = concat_seq + padding
        df.at[index, 'sequence'] = [seq_map[s] for s in concatenated_seq]
        #react_columns.at[index, 'maped_sequence'] = [seq_map[s] for s in concatenated_seq]


In [ ]:

if data_preprocessing_done == 0:
    #react_columns['reactivity_0207'] = copy.deepcopy(react_columns['reactivity_0206'])
    #react_columns['reactivity_0208'] = copy.deepcopy(react_columns['reactivity_0206'])
    react_columns['mapped_sequence'] = copy.deepcopy(df['sequence'])
    #del react_columns['reactivity_0207']
    #del react_columns['reactivity_0208']
if save_pkl == 1:
    react_columns.to_pickle("react_columns.pkl")

if load_pkl == 1:
    react_columns = pd.read_pickle("react_columns.pkl")
    


In [ ]:
# Define a custom dataset class for your data
class CustomDataset(Dataset):
    def __init__(self, data, max_seq_len):
        self.data = data
        self.max_seq_len = max_seq_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        trg_sequence = torch.tensor(self.data.iloc[idx, :].tolist()[:-1])
        src_sequence = torch.tensor(self.data.iloc[idx, :].tolist()[-1])
        
        #print("Size src seq ", src_sequence.size())
        #print("Size trg seq ", trg_sequence, trg_sequence.size())
        
        return src_sequence, trg_sequence

    
# Create a DataLoader for batching and shuffling
batch_size = 64
#batch_size = 3


# Define the size of the training set
dataset_size = len(react_columns)
train_size = int(0.5 * dataset_size)
test_size = dataset_size - train_size

custom_dataset = CustomDataset(react_columns, max_seq_len)
train_indices, test_indices = random_split(custom_dataset, [train_size, test_size])
train_dataloader = DataLoader(train_indices, batch_size=batch_size)
test_dataloader = DataLoader(test_indices, batch_size=batch_size)


    

In [ ]:
print(device)


class MyTransformerEncoderLayer(nn.TransformerEncoderLayer):
    def __init__(self, d_model, nhead, batch_first=True, **kwargs):
        super(MyTransformerEncoderLayer, self).__init__(d_model, nhead, **kwargs)
        self.self_attn = nn.MultiheadAttention(d_model, nhead, batch_first=batch_first)


class TransformerRegressionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, nhead, num_layers, output_sequence_length):
        super(TransformerRegressionModel, self).__init__()
        
        self.embedding = nn.Embedding(input_dim, hidden_dim)
        '''
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=input_dim, nhead=nhead),
            num_layers=num_layers
        )
        '''
        self.transformer_encoder = nn.TransformerEncoder(
            MyTransformerEncoderLayer(d_model=hidden_dim, nhead=nhead, batch_first=True),
            num_layers=num_layers
        )
        
        
        
        self.transformer_decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model=input_dim, nhead=nhead),
            num_layers=num_layers
        )
        
        
        
        
        self.fc = nn.Linear(input_dim, output_sequence_length * 1)
        self.gelu = nn.GELU()
        self.output_sequence_length = output_sequence_length


    
    def forward(self, x):       
        # Apply embedding to input
        #print("X shape 1", x.shape)
        x = self.embedding(x)
        #print("X shape 2", x.shape)
        # Permute to (sequence_length, batch_size, input_dim)
        x = x.permute(1, 0, 2)
        #print("X shape 3", x.shape)
        
        # Encoder forward pass
        encoder_output = self.transformer_encoder(x)

        # Decoder forward pass with memory from the encoder
        decoder_output = self.transformer_decoder(x, memory=encoder_output)

        # Global average pooling over the sequence dimension
        x = decoder_output.mean(dim=0)

        # Final fully connected layer with GELU activation
        x = self.gelu(self.fc(x))

        return x
    

# Hyperparameters
input_dim =  max_seq_len # Adjust based on your input size
hidden_dim = 208
nhead = 4
num_layers = 3
output_sequence_length = max_seq_len - 2  # Adjust based on your output sequence length
#batch_size = 1
num_epochs = 2
learning_rate = 0.001


# Instantiate the model, loss function, and optimizer
model = TransformerRegressionModel(input_dim, hidden_dim, nhead, num_layers, output_sequence_length).to(device)
criterion = nn.MSELoss()
#optimizer = optim.Adam(model.parameters(), lr=learning_rate)

optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9, weight_decay=0, amsgrad=False)



training_losses = []  # to store training losses
validation_losses = []
prev_average_val_loss = 10000000000
list_of_avg_loss = []
epochss = []
# Training loop
for epoch in range(num_epochs):
    epoch_loss = 0.0  # accumulate loss for the entire epoch
    val_loss = 0.0
    #for inputs, targets in tqdm(train_dataloader.iterrows(), total=len(train_dataloader), desc="Processing rows"):
    #with tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as tqdm_loader:
    #    for inputs, targets in tqdm_loader:
         
    model.train()        
    #for batch, dat in enumerate(train_dataloader, 1): 
    for batch, dat in enumerate(tqdm(train_dataloader, desc="Processing batches", unit="batch"), 1):
            inputs = dat[0].to(device, dtype=torch.long)
            targets = dat[1].to(device).float()
            optimizer.zero_grad()
            outputs = model(inputs)
            #print(f'TRAIN:  Batch {batch}, Input: {inputs.shape}, Outputs: {outputs.shape}, Targets: {targets.shape}')
            outputs[outputs < 0] = 0.0
            outputs[outputs > 1] = 1.0
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            #print(f"Epoch: {epoch+1}, Training Loss: {loss.item()}")
    
    model.eval()
    with torch.no_grad():
        #for batch, dat in enumerate(test_dataloader, 1):
        for batch, dat in enumerate(tqdm(test_dataloader, desc="Processing batches", unit="batch"), 1):
            inputs = dat[0].to(device, dtype=torch.long)
            targets = dat[1].to(device).float()
            output = model(inputs)
            #print(f'VAL:  Batch {batch}, Input: {inputs.shape}, Outputs: {output.shape}, Targets: {targets.shape}')
            loss = criterion(output, targets)
            print(f"Epoch: {epoch+1}, Validation Loss: {loss.item()}")
            val_loss += loss.item()
    
    #outputs.requires_grad_(True)
    
    
    average_epoch_loss = epoch_loss / len(train_dataloader)
    training_losses.append(average_epoch_loss)
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {average_epoch_loss:.4f}')
    
    average_val_loss = val_loss / len(test_dataloader)
    validation_losses.append(average_val_loss)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {average_val_loss:.4f}')
    epochss.append(epoch)
    
    list_of_avg_loss.append(average_val_loss)
    #Save best model Every Epoch
    if average_val_loss < prev_average_val_loss:
        torch.save(model, "Trained_Model.model")
        print("Model Saved because ")
        print(f'Current average validation loss {average_val_loss:.4f} is less than previous average validation loss {prev_average_val_loss:.4f}')
    prev_average_val_loss = min(list_of_avg_loss)


# Plotting the training loss
plt.plot(epochss, training_losses, label='Training Loss')
plt.plot(epochss, validation_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()
plt.show()
# After training, you can use the model for prediction
# Example usage:
# test_input = torch.randn(1, sequence_length, input_dim)  # Adjust based on your input size and sequence length
# predicted_output = model(test_input)


In [ ]:
#Validation predict
#val_df = 

#Read TEST Sequence
test_df = pd.read_csv('/media/spartans/COMMON:/RNA_Project/test_sequences.csv')

In [ ]:
test_df.head(4)
test_df.at[0, 'sequence']

In [ ]:
len(test_df['sequence'][0])

In [ ]:
test_df['DMS_sequence'] = copy.deepcopy(test_df['sequence'])
test_df['2A3_sequence'] = copy.deepcopy(test_df['sequence'])


test_df_sq = copy.deepcopy(test_df['sequence']) #saving a backup of sequences.
#react_columns['maped_sequence'] = [' ']*len(react_columns)
#for index, row in tqdm(df.iterrows(), total=len(df.rows), desc="Processing rows"):
for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Processing rows"):
    
    padding = "Z"*(max_seq_len - 2 - len(test_df.at[index, 'sequence']))
    form_DMS_seq = 'M' + test_df.at[index, 'sequence'] + 'N' + padding
    form_2A3_seq = 'T' + test_df.at[index, 'sequence'] + 'X' + padding
    

    test_df.at[index, 'DMS_sequence'] = [seq_map[s] for s in form_DMS_seq]
    test_df.at[index, '2A3_sequence'] = [seq_map[s] for s in form_2A3_seq]
    if small_dataset == 1:
        break
    

In [ ]:
#torch.no_grad():
#src_data = src_data.to(device=device)
#tgt_data = tgt_data.to(device=device, dtype=torch.long)
#output = transformer_model(src_data, tgt_data[:, :-1])

#out = transformer_model(test_df['2A3_sequence'][0], tgt_data[:, :-1])

input_sequence = torch.tensor([test_df.at[0, 'DMS_sequence']]).to(device=device, dtype=torch.long)
#inputs = dat[0].to(device, dtype=torch.long)
out = []
print("input seq", input_sequence.size())
out = model(input_sequence)
print("out", out.size)
out[out < 0] = 0.0
out

In [ ]:
test_df['id_max'][0]


for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Processing rows"):
    input_sequence = torch.tensor([test_df.at[index, 'DMS_sequence']])
    seq_length = len(test_df.at[index, 'sequence'])
    out_dms = []
    out_dms = model(input_sequence).tolist()
    out_dms[:seq_length]
   
    out_2a3 = []
    input_sequence = torch.tensor([test_df.at[index, '2A3_sequence']])
    out_2a3 = model(input_sequence).tolist()
    out_2a3[:seq_length]
    
    
    
    

In [ ]:
aa = out.tolist()
#test_df['id_min'][0]
aa[:5]

In [ ]:
ii = test_df.shape[0]   - 1
print(ii)
#for ii in range(1643450, 1643670):
    #ii = 1643680
seq_length = len(test_df.at[ii, 'sequence'])
print(seq_length, test_df['id_min'][ii], test_df['id_max'][ii],  test_df['id_max'][ii] - test_df['id_min'][ii])
